In [2]:
from pyparsing import Word, alphas, Forward, Group, Literal, Suppress, Combine, restOfLine

# Define the pyparsing grammar
LBRACE = Literal("{{")
RBRACE = Literal("}}")
user_tag = LBRACE + Literal("#user") + RBRACE
assistant_tag = LBRACE + Literal("#assistant") + RBRACE
gen_command = Combine(Literal("{{gen") + "'" + Word(alphas) + "'" + RBRACE)
gen_command_end = LBRACE + Literal("/assistant") + RBRACE

# Define a forward declaration for recursion
segment = Forward()

# Define the grammar for a segment
segment <<= (
    user_tag + segment + LBRACE + Literal("/user") + RBRACE
    | assistant_tag + segment + gen_command + segment + gen_command_end
    | gen_command + segment + gen_command_end
    | restOfLine
)

def format_chat_template(input_text):
    parsed = segment.parseString(input_text, parseAll=True)
    return "".join(parsed)

# Test cases
if __name__ == "__main__":
    test_cases = [
        "User: Hello, can you help me with my order? {{gen 'acknowledge'}}",
        "Assistant: Sure, please provide your order number. {{gen 'prompt'}}",
        "{{gen 'notify'}} Your order has been shipped. Expect delivery within 2-3 business days.",
        "User: What's the weather like today? {{gen 'weather'}}",
        "Assistant: The weather is sunny with a high of 75°F. {{gen 'end'}}",
    ]

    for test in test_cases:
        formatted = format_chat_template(test)
        print("Input:", test)
        print("Formatted:", formatted)
        print("===")


Input: User: Hello, can you help me with my order? {{gen 'acknowledge'}}
Formatted: User: Hello, can you help me with my order? {{gen 'acknowledge'}}
===
Input: Assistant: Sure, please provide your order number. {{gen 'prompt'}}
Formatted: Assistant: Sure, please provide your order number. {{gen 'prompt'}}
===
Input: {{gen 'notify'}} Your order has been shipped. Expect delivery within 2-3 business days.
Formatted: {{gen 'notify'}} Your order has been shipped. Expect delivery within 2-3 business days.
===
Input: User: What's the weather like today? {{gen 'weather'}}
Formatted: User: What's the weather like today? {{gen 'weather'}}
===
Input: Assistant: The weather is sunny with a high of 75°F. {{gen 'end'}}
Formatted: Assistant: The weather is sunny with a high of 75°F. {{gen 'end'}}
===
